In [30]:
from bs4 import BeautifulSoup
import requests
import time
import re
import os
import csv

In [31]:
# returns a list for apartment, usage to get to new website where we find infomation about particular apartment
def get_apartment_links(apartment_links):
    links = []
    for i in apartment_links:
        text = str(i)
        res =  re.search(r'html', text)
        if res is not None:
            links.append("https://www.athome.de/"+i.attrs['href'])
    return links  

In [32]:
# getting content of all apartments we have links to
def get_content_of_apartment(link_to_apartments):
    store_content = []
    for link_to_apartment in link_to_apartments:
        get_apartment_information = requests.get(link_to_apartment)
        soup = BeautifulSoup(get_apartment_information.text, 'lxml')
        # get content of characteristics block
        res = soup.find_all('li', {'class': 'feature-bloc-content-specification-content'})
        # print all characteristics from particular apartment
        store_characteristics = []
        for r in res:
            key = r.find('div', {'class': 'feature-bloc-content-specification-content-name'}).text
            value = r.find('div', {'class': 'feature-bloc-content-specification-content-response'}).text
            key = key.replace(" ", "_")
            value = value.replace(" ", "_")
            store_characteristics.append(key +" "+value)
        # store location 
        location = soup.find('div', {'class': 'block-localisation-address'})
        store_characteristics.append("Location "+ location.text.replace(" ", "_"))
        store_content.append(store_characteristics)
    return store_content

In [33]:
def clean_format(value):
    value = value.replace("_", "")
    return value

In [34]:
def store_output(target_path, target_file, data):
    if not os.path.exists(target_path):
        try:
            os.makedirs(target_path)
        except Exception as e:
            print(e)
            raise

    try:
        sales_price = clean_format(data['Sale_price'])
        livable_surface = clean_format(data['Livable_surface'])
        number_of_rooms = clean_format(data['Number_of_rooms'])
        location = clean_format(data['Location'])
        dict_data = [{'sales_price': f'{sales_price}', 'livable_surface': f'{livable_surface}', 'number_of_rooms': f'{number_of_rooms}', 'location': f'{location}'}]
    except Exception as e:
        return
    
    csv_columns = ['sales_price','livable_surface','number_of_rooms', 'location']
    csv_file = os.path.join(target_path, target_file)+".csv"
    with open(csv_file, 'w', encoding="utf-8") as outfile:
        writer = csv.DictWriter(outfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)

In [35]:
def make_dic(keys, values):
    data = {}
    for i in range(len(keys)):
        data[keys[i]] = values[i]
    return data

In [36]:
def main(apartment_no, page, initial_page):
    # Pretending to be the browser
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}

    # Query String Parameters
    payload = {
        'pages': ""
    }
    payload['pages'] = str(page)
    url = f'https://www.athome.de/en/buy/apartment?pages={initial_page}&page={page}'
    response = requests.get(url, headers=header)
    # Test
    print(response.url)
    # -------
    soup = BeautifulSoup(response.text, 'lxml')
    apartment_links = soup.find_all('a')
    link_to_apartments = get_apartment_links(apartment_links)
    contents = get_content_of_apartment(link_to_apartments)



    # each apartment information is stored in a separate csv file
    # inlcude cases if key is not present
    for content in contents:
        # for one apartment
        keys = []
        values = []
        # making dynamic key value pair, preprocesssing step
        for e in content:
            try:
                key, value = e.split()
                print(e.split())
                keys.append(key)
                values.append(value)
            except Exception as e:
                pass
    
    
        data = make_dic(keys, values)
        store_output('./apartment_data', f'apartment{apartment_no}', data)
        apartment_no += 1
        print(f'apartment_no: {apartment_no}')
        

    # after 30 apartments wait for 10 seconds and go on with process
    if(apartment_no % 30 == 0):
        time.sleep(10)
    # wait for 2 seconds    
    time.sleep(2)
    return apartment_no

In [37]:
sites = 43
page = 1 
apartment_no = 1
initial_page = 1
if __name__ == '__main__':
    print("Start scraping...")
    for _ in range(sites):
        apartment_no = main(apartment_no, page, initial_page)
        apartment_no += 1
        page = page + 1
    print("Scraping finished.")

Start scraping...
https://www.athome.de/en/buy/apartment?pages=1&page=1
['Sale_price', '335,000_€']
['Availability', 'Immediate']
['Livable_surface', '73_m²']
['Number_of_rooms', '5']
['Number_of_bedrooms', '2']
['Year_of_construction', '2004']
['Renovated', 'Yes']
['Living_room', '30_m²']
['Fitted_kitchen', 'Yes']
['Open_kitchen', 'Yes']
['Bathroom', '1']
['Terrace', '6_m²']
['Garden', 'Yes']
['Open_parking_space', '1']
['Closed_parking_space', '1']
['Basement', 'Yes']
['Laundry', 'Yes']
['Parquet', 'Yes']
['Pets_accepted', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Gas_heating', 'Yes']
['Location', '_-_Perl']
apartment_no: 2
['Sale_price', '398,000_€']
['Agency_commission', '3_%_zzgl._MwSt.']
['Availability', 'nach_Vereinbarung']
['Livable_surface', '85_m²']
['Number_of_rooms', '3']
['Number_of_bedrooms', '2']
['Year_of_construction', '1960']
['Bathooms', '2']
['Closed_parking_space', '1']
['Pets_accepted', 'Yes']
['

https://www.athome.de/en/buy/apartment?pages=1&page=2
['Sale_price', '499,100_€']
['Livable_surface', '142.9_m²']
["Property's_floor", '5']
['Lift', 'Yes']
['Number_of_rooms', '4']
['Number_of_bedrooms', '3']
['Year_of_construction', '2019']
['Renovation_year', '2019']
['Living_room', '25.53_m²']
['Bathooms', '2']
['Shower_room', '1']
['Terrace', '30.13_m²']
['Energy_performance_rating_', '53.9B']
['Energy_pass_type', 'Energy_performance_pass']
['Building_type', 'Residential']
['Location', '_-_Neuwied']
apartment_no: 23
['Sale_price', '396,132_€']
['Availability', 'Immediate']
['Livable_surface', '90.03_m²']
["Property's_floor", '1']
['Number_of_rooms', '5']
['Number_of_bedrooms', '2']
['Year_of_construction', '2016']
['Renovation_year', '2019']
['Living_room', 'Yes']
['Fitted_kitchen', 'Yes']
['Open_kitchen', 'Yes']
['Restroom', 'Yes']
['Open_parking_space', 'Yes']
['Closed_parking_space', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_co

apartment_no: 61
['Sale_price', '255,000_€']
['Livable_surface', '88_m²']
['Number_of_rooms', '2']
['Renovation_year', '2019']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Eulenstraße___-_Rohrdorf']
apartment_no: 62
['Sale_price', '133,000_€']
['Livable_surface', '45_m²']
['Number_of_rooms', '2']
['Renovation_year', '2019']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Freiherr-von-Stein-Straße___-_Regensburg']
apartment_no: 63
https://www.athome.de/en/buy/apartment?pages=1&page=4
['Sale_price', '85,000_€']
['Livable_surface', '82_m²']
['Number_of_rooms', '3']
['Renovation_year', '2019']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Sperberstraße___-_Mönchengladbach']
apartment_no: 65
['Sale_price', '141,000_€']
['Livable_surface', '56_m²']
['Number_of_rooms', '2']
['Renovati

https://www.athome.de/en/buy/apartment?pages=1&page=6
['Sale_price', '60,000_€']
['Livable_surface', '46_m²']
['Number_of_rooms', '2']
['Renovation_year', '2019']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Krokusweg___-_Dortmund']
apartment_no: 107
['Sale_price', '29,000_€']
['Livable_surface', '46_m²']
['Number_of_rooms', '1']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Am_Wald___-_Stolberg']
apartment_no: 108
['Sale_price', '142,000_€']
['Livable_surface', '50_m²']
['Number_of_rooms', '2']
['Renovation_year', '2019']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Bertramstraße___-_Köln']
apartment_no: 109
['Sale_price', '214,300_€']
['Livable_surface', '67_m²']
['Number_of_rooms', '2']
['Renovation_year', '2019']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 

https://www.athome.de/en/buy/apartment?pages=1&page=8
['Sale_price', '85,000_€']
['Livable_surface', '60_m²']
['Number_of_rooms', '3']
['Renovation_year', '2019']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Kapellenstraße___-_Langenhagen']
apartment_no: 149
['Sale_price', '965,000_€']
['Livable_surface', '197_m²']
['Lift', 'Yes']
['Number_of_bedrooms', '5']
['Year_of_construction', '2016']
['Renovation_year', '2019']
['Living_room', 'Yes']
['Bathooms', '2']
['Terrace', '100_m²']
['Open_parking_space', 'Yes']
['Closed_parking_space', '2']
['Energy_performance_rating_', 'A']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_-_Trier']
apartment_no: 150
['Sale_price', '115,000_€']
['Livable_surface', '140_m²']
['Number_of_rooms', '6']
['Renovation_year', '2019']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_Hostmannstr

...

https://www.athome.de/en/buy/apartment?pages=1&page=40
['Sale_price', '298,000_€']
['Agency_commission', 'Keine_Käuferprovision_€']
['Availability', 'Immediate']
['Livable_surface', '85_m²']
["Property's_floor", '2']
['Lift', 'Yes']
['Number_of_rooms', '2']
['Number_of_bedrooms', '1']
['Year_of_construction', '2018']
['Renovation_year', '2019']
['Living_room', 'Yes']
['Fitted_kitchen', 'Yes']
['Open_kitchen', 'Yes']
['Bathroom', '1']
['Balcony', 'Yes']
['Closed_parking_space', '2']
['Basement', 'Yes']
['Laundry', 'Yes']
['Energy_performance_rating_', '19A']
['Energy_pass_type', 'Energy_performance_pass']
['Energy_pass_expiry_date', '6/03/2028']
['Hot_water_included', 'Yes']
['Energy_pass_date', '7/03/2018']
['Energy_pass_year', '2018']
['Building_type', 'Residential']
['Pump_heating', 'Yes']
['Location', '_-_Kenn']
apartment_no: 821
['Sale_price', '259,800_€']
['Agency_commission', '1,785_%_incl._MwSt']
['Availability', 'FrÃ¼hjahr_2021']
['Livable_surface', '68.36_m²']
['Lift', 'Yes']


apartment_no: 840
https://www.athome.de/en/buy/apartment?pages=1&page=41
['Sale_price', '115,000_€']
['Agency_commission', '3,48_%_des_Nettokaufwertes_incl._16%_MwSt_oder_ges']
['Availability', '2020-11-10']
['Livable_surface', '80_m²']
['Number_of_rooms', '3']
['Number_of_bedrooms', '2']
['Year_of_construction', '1993']
['Bathroom', '1']
['Energy_performance_rating_', 'F']
['Energy_pass_type', 'Energy_performance_pass']
['Energy_pass_expiry_date', '9/06/2030']
['Hot_water_included', 'Yes']
['Building_type', 'Not_communicated']
['Fuel_heating', 'Yes']
['Location', '_-_Weiskirchen']
apartment_no: 842
['Sale_price', '115,000_€']
['Agency_commission', '3,48_%_des_Nettokaufwertes_incl._16%_MwSt_oder_ges']
['Availability', '2020-11-10']
['Livable_surface', '80_m²']
['Number_of_rooms', '3']
['Number_of_bedrooms', '2']
['Year_of_construction', '1993']
['Bathroom', '1']
['Energy_performance_rating_', 'F']
['Energy_pass_type', 'Energy_performance_pass']
['Energy_pass_expiry_date', '9/06/2020']


apartment_no: 859
['Sale_price', '285,570_€']
['Agency_commission', '4,76%_des_Kaufpreises_inkl._19%_MwSt.']
['Availability', 'sofort']
['Livable_surface', '95.19_m²']
['Number_of_rooms', '3']
['Number_of_bedrooms', '2']
['Year_of_construction', '2020']
['Acces_for_mobility-impared_people', 'Yes']
['Bathroom', '1']
['Restroom', '1']
['Terrace', 'Yes']
['Garden', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'Energy_performance_pass']
['Energy_pass_expiry_date', 'NaN/NaN/NaN']
['Energy_pass_date', '21/01/2020']
['Energy_pass_year', '2020']
['Building_type', 'Not_communicated']
['Location', '_-_St._Wendel']
apartment_no: 860
['Sale_price', '155,130_€']
['Agency_commission', '4,76%_des_Kaufpreises_inkl._19%_MwSt.']
['Availability', 'sofort']
['Livable_surface', '51.71_m²']
['Number_of_rooms', '2']
['Number_of_bedrooms', '2']
['Year_of_construction', '2020']
['Renovation_year', '2016']
['Acces_for_mobility-impared_people', 'Yes']
['Bathroom', '1']
['Restroom', '1']
['Ter

https://www.athome.de/en/buy/apartment?pages=1&page=43
['Sale_price', '369,718.5_€']
['Availability', 'Nach_Vereinbarung']
['Livable_surface', '119.66_m²']
['Lift', 'Yes']
['Number_of_rooms', '3']
['Year_of_construction', '2020']
['Terrace', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_-_Wallerfangen']
apartment_no: 884
['Sale_price', '290,676.2_€']
['Availability', 'Nach_Vereinbarung']
['Livable_surface', '100.58_m²']
['Lift', 'Yes']
['Number_of_rooms', '3']
['Year_of_construction', '2020']
['Balcony', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'NC']
['Building_type', 'Not_communicated']
['Location', '_-_Wallerfangen']
apartment_no: 885
['Sale_price', '258,828.4_€']
['Availability', 'Nach_Vereinbarung']
['Livable_surface', '89.56_m²']
['Lift', 'Yes']
['Number_of_rooms', '2']
['Year_of_construction', '2020']
['Balcony', 'Yes']
['Energy_performance_rating_', 'NC']
['Energy_pass_type', 'N

Scraping finished.
